In [54]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
# from groupby_toolz import enrich_db, gcloud
# pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [55]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]    
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

In [56]:
customer_id = '90'
customer_name='%wbmason%'
buckets = """Multiple Buckets"""
strategy_version_input=682
attribute_id_input=2443
formatted_attribute = 'height'

In [57]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [58]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]

In [59]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df))
df = df[df['bucket_id'].isin(buckets)]
len(df)

17668


17668

In [60]:
height=df
regex_pattern_general=r'(?i)(\b\d+\.\d*.?\"\sH)|(\b\d+.?\"\sH)|(\b\d+\.\d*.?\"\sH)|(\d*\s?\-\d*\/\d*\\\"\sH)|(\d*\s\-?\d*\/\d*\\\"\sH)|(\d+\.?\d*''\sH)|()'
# regex_pattern_general=r'(?i)(\b\d+\.\d*.?\"\s?H)|(\b\d+.?\"\s?H)|(\b\d+\.\d*.?\"\s?H)|(\d*\s?\-\d*\/\d*\\\"\s?H)|(\d*\s\-?\d*\/\d*\\\"\s?H)|(\d+\.?\d*''\s?H)|()'
height['matches'] = height['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
height['name_matches'] = height['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
height['matches']= height['matches'].apply(lambda x: sorted(x, key=natural_key))
print(len(height))
print(len(height[height['matches'].astype(str)=='[]']))
print(len(height[height['matches'].astype(str)!='[]']))
# (\s\d+\.\d*.?\"\s?H\s)|(\s\d+.?\"\s?H)

17668
17006
662


In [61]:
height[height['matches'].astype(str)=='[]']['buckets'].explode().value_counts()[0:30]

Envelopes/Mailers                                   6563
Mailing & Packing Boxes                             3807
Protective Packing Materials                        1678
Office Chairs                                        773
Desks/Workstations/Workbenches                       662
Chairs                                               576
Drawers                                              452
Table Replacement Parts & Accessories                361
Cabinets                                             273
Stools                                               253
Furniture Variety Packs                              163
Folding Tables & Tray Tables                         154
Temporary Seating                                    141
Boxes, Baskets, & Bins                               125
Dividers/Partitions/Screens                          108
Utility Tables                                        93
Furniture Accessories & Replacement Parts             88
Dining Tables                  

In [73]:
packing=height[height['matches'].astype(str)=='[]'][height['buckets'].astype(str)=='Protective Packing Materials']
regex_pattern_packing=r'(?i)(tubes)|()'
# tubes=Boxes
# tubes['matches']=tubes['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# not_tube=tubes[tubes['matches'].astype(str)=='[]']
# not_tube[0:500]
packing[0:500]

product_id external_id  \
1514   434506557  5637487283   
1515   434476825  5637448715   
1628   675184936  5637544566   
1629   675184935  5637544565   
1630   675184933  5637544563   
1631   675184931  5637544561   
1833   675184928  5637544558   
1834   675184926  5637544556   
1835   675184923  5637544553   
1837   675184922  5637544552   
1838   675183394  5637544544   
1839   675183393  5637544543   
1855   434471868  5637442009   
1857   434471864  5637442005   
1862   434471856  5637441998   
1868   675183390  5637544540   
1869   675183389  5637544539   
1870   675183388  5637544538   
1871   675184917  5637544537   
1872   675184916  5637544536   
1873   675184915  5637544535   
1874   675184914  5637544534   
1875   675184913  5637544533   
1876   675184912  5637544532   
1877   675184910  5637544530   
1878   675184908  5637544528   
1879   675184905  5637544525   
1880   675184903  5637544523   
1881   675184897  5637544517   
1882   675184895  5637544515   
1883   675184894  5637544514   
1884   675184893  5637544513   
1885   675184890  5637544510   
1886   675184889  5637544509   
1887   675184887  5637544507   
1888   675184885  5637544505   
1889   675184882  5637544502   
1890   675184881  5637544501   
1891   675184880  5637544500   
1897   675180393  5637545591   
1898   675180392  5637545590   
2008   434493163  5637467078   
2033   675180320  5637545496   
2034   675180318  5637545494   
2035   675180306  5637545482   
2036   675180304  5637545480   
2037   675180302  5637545478   
2038   675180294  5637545470   
2039   675180293  5637545469   
2040   675184567  5637545468   
2041   675183483  5637545466   
2042   675183482  5637545465   
2043   675183480  5637545463   
2044   675183474  5637545457   
2045   675184564  5637545454   
2046   675184561  5637545451   
2047   675184559  5637545449   
2048   675184552  5637545442   
2087   675180326  5637545502   
2088   675180322  5637545498   
2164   434494757  5637463853   
2165   434494754  5637463849   
2169   434471712  5637440516   
2263   434493791  5637463141   
2264   434493788  5637463138   
2265   434493787  5637463137   
2266   434493784  5637463134   
2267   434493781  5637463131   
2268   434493780  5637463130   
2269   434493779  5637463129   
2270   434493771  5637463119   
2277   434471845  5637441988   
2329   434493835  5637463187   
2330   434493834  5637463186   
2331   434493833  5637463185   
2335   434493813  5637463164   
2336   434493812  5637463163   
2337   434493810  5637463161   
2338   434493809  5637463160   
2339   434493805  5637463155   
2340   434493804  5637463153   
2341   434493801  5637463150   
2342   434493799  5637463148   
2343   434493797  5637463147   
2344   434493795  5637463145   
2345   434493792  5637463142   
2391   434473872  5637451140   
2392   434473870  5637451138   
2393   434473869  5637451137   
2394   434474920  5637451136   
2403   434494643  5637463706   
2404   434494624  5637463667   
2405   434494623  5637463666   
2406   434494622  5637463665   
2408   434495811  5637463615   
2410   434494717  5637463780   
2411   434494711  5637463774   
2412   434494710  5637463773   
2413   434494709  5637463772   
2424   594918034  5637528204   
2443   434442795  5637395374   
2444   434474916  5637451132   
2456   592697127  5637527061   
2467   434387154  5637213097   
2528   434502257  5637481471   
2530   434474875  5637451090   
2531   434474873  5637451088   
2546   434504894  5637481087   
2547   434504891  5637481085   
2548   434504890  5637481084   
2549   434504889  5637481083   
2550   434465449  5637429780   
2551   434465445  5637429776   
2552   434465444  5637429775   
2553   434465443  5637429774   
2554   434465442  5637429773   
2555   434465441  5637429772   
2556   434465440  5637429771   
2557   434465438  5637429770   
2558   434465437  5637429769   
2559   434465436  5637429768   
2560   434465435  5637429767   
2561   434465434  5637429766   
2562   434465431  5637429763   
2563  

In [71]:
Boxes=height[height['matches'].astype(str)=='[]'][height['buckets'].astype(str)=='Mailing & Packing Boxes']
regex_pattern=r'(?i)(tubes)|()'
tubes=Boxes
tubes['matches']=tubes['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
not_tube=tubes[tubes['matches'].astype(str)=='[]']
not_tube[0:500]
tube diam

product_id external_id  \
115    507120861  5637511654   
285    434514666  5637497740   
289    434513395  5637499524   
433    434513398  5637499527   
500    507120452  5637511649   
857    434502239  5637481452   
980    434463320  5637428651   
985    434463327  5637428658   
986    434463315  5637428646   
987    434463318  5637428649   
988    434463330  5637428661   
989    434463326  5637428657   
990    434463314  5637428645   
991    434463319  5637428650   
994    434462434  5637428716   
1087   434463311  5637428642   
1262   434463328  5637428659   
1431   675180485  5637544010   
1432   675180479  5637544004   
1433   675180477  5637544002   
1434   675183614  5637543998   
1435   675183612  5637543996   
1436   675183606  5637543990   
1437   675183604  5637543988   
1438   675183602  5637543986   
1439   675183600  5637543984   
1440   675183596  5637543980   
1441   675183594  5637543978   
1442   675183592  5637543976   
1443   675183590  5637543974   
1444   675183588  5637543972   
1445   675183586  5637543970   
1446   675183584  5637543968   
1447   675183578  5637543962   
1448   675183576  5637543960   
1449   675183574  5637543958   
1450   675183572  5637543956   
1451   675183568  5637543952   
1452   675183566  5637543950   
1453   675183564  5637543948   
1454   675183562  5637543946   
1455   675183560  5637543944   
1456   675183558  5637543942   
1457   675183556  5637543940   
1470   675184474  5637545364   
1471   675184473  5637545363   
1472   675184472  5637545362   
1473   675184471  5637545361   
1474   675184467  5637545357   
1475   675184466  5637545356   
1476   675184465  5637545355   
1477   675184463  5637545353   
1478   675184458  5637545348   
1513   434505898  5637487462   
1521   675183656  5637544062   
1522   675183654  5637544060   
1523   675183652  5637544058   
1524   675183648  5637544054   
1525   675180489  5637544044   
1526   675180487  5637544042   
1528   675183643  5637544038   
1529   675183637  5637544032   
1530   675183635  5637544030   
1531   675183627  5637544022   
1532   675183625  5637544020   
1533   675183623  5637544018   
1534   675183621  5637544016   
1535   675183617  5637544012   
1550   675184667  5637543538   
1551   675184663  5637543534   
1552   675184659  5637543530   
1553   675184655  5637543526   
1554   675184647  5637543518   
1555   675184645  5637543516   
1556   675184643  5637543514   
1558   675184635  5637543506   
1559   675184633  5637543504   
1561   675184623  5637543494   
1562   675184621  5637543492   
1563   675184615  5637543486   
1564   675184613  5637543484   
1565   675184611  5637543482   
1566   675184607  5637543478   
1567   675184601  5637543472   
1573   675184589  5637543460   
1574   675184587  5637543458   
1598   675180735  5637544649   
1599   675180733  5637544647   
1600   675180731  5637544645   
1601   675180725  5637544639   
1602   675180719  5637544633   
1603   675180717  5637544631   
1604   675180715  5637544629   
1605   675180713  5637544627   
1606   675180711  5637544625   
1607   675180709  5637544623   
1608   675180707  5637544621   
1609   675180703  5637544617   
1610   675180701  5637544615   
1611   675180697  5637544611   
1612   675184966  5637544607   
1613   675184964  5637544605   
1614   675184962  5637544603   
1615   675184960  5637544601   
1616   675184956  5637544597   
1617   675184952  5637544593   
1618   675184950  5637544591   
1619   675184948  5637544589   
1620   675184946  5637544587   
1621   675183407  5637544585   
1622   675183405  5637544583   
1623   675183403  5637544581   
1624   675183401  5637544579   
1625   675184945  5637544575   
1626   675184941  5637544571   
1627   675184939  5637544569   
1725   675180747  5637544661   
1726   675180739  5637544653   
1727   434506635  5637487794   
1728   675180737  5637544651   
1729   434506502  5637487227   
1731   434506477  5637487201   
1826   434422196  5637353987   
1893   434434222  5637377588   
1894  

In [63]:
envelope=height[height['matches'].astype(str)=='[]'][height['buckets'].astype(str)=='Envelopes/Mailers']
regex_pattern_general=r'(?i)(\d+.?\d+\/\d+.?\s?x\s?\d+\s?\d+\/\d+.?)|(\d+\.\d+\s?x\s?\d+\.\d+)|(\d+\.?\d*\s?x\s?\d+\.?\d*)|(\d+.?\d+\/\d+.?\s?x\s?\d+\.?\d*.?)|(\d+\.?\d*.?\s?x\s?\d+.?\d+\/\d+.?)|(\d+.?\s?x\s?\d+.?)|()'
envelope['envelope_matches']=envelope['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
envelope['envelope_name_matches']=envelope['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))

In [65]:
envelope[0:500]

product_id external_id  \
379    434461803  5637428456   
380    434461802  5637428455   
1461   434403195  5637282994   
1462   434403188  5637282987   
1463   434413029  5637322091   
1479   675184456  5637545346   
1480   675184454  5637545344   
1481   675184450  5637545340   
1482   675184449  5637545339   
1483   675184447  5637545337   
1484   675184446  5637545336   
1485   675184445  5637545335   
1486   675184444  5637545334   
1487   675184442  5637545332   
1488   675184440  5637545330   
1489   675184439  5637545329   
1490   675184438  5637545328   
1491   675184436  5637545326   
1492   675184434  5637545324   
1493   675184431  5637545321   
1494   675184429  5637545319   
1495   675184426  5637545316   
1496   675184419  5637545309   
1497   675184418  5637545308   
1498   675184417  5637545307   
1512   434505036  5637487481   
1516   434410447  5637324684   
1517   434410446  5637324683   
1519   434403314  5637283900   
1527   434519997  5637283243   
1536   434403237  5637283208   
1538   434496493  5637475112   
1539   540455808  5637521558   
1540   540455807  5637521557   
1541   434496487  5637475106   
1542   434497269  5637475105   
1543   434497268  5637475104   
1544   434497265  5637475101   
1545   434498000  5637475098   
1546   434497997  5637475095   
1557   434497996  5637475094   
1560   434497995  5637475093   
1595   434411106  5637320917   
1597   434400822  5637282249   
1634   434518836  5637501732   
1635   434518834  5637501729   
1636   434518833  5637501728   
1637   434518830  5637501725   
1638   434518829  5637501724   
1641   535711568  5637521201   
1642   535711566  5637521200   
1643   535711561  5637521194   
1644   434518831  5637501726   
1645   535711521  5637521191   
1646   535711520  5637521190   
1647   535711519  5637521189   
1648   535711518  5637521188   
1649   535711578  5637521180   
1650   535711573  5637521149   
1651   535711571  5637521148   
1652   535720189  5637521145   
1653   434486083  5637169652   
1654   434486081  5637169650   
1655   434486854  5637169557   
1656   434486932  5637169649   
1657   434486925  5637169641   
1658   434486924  5637169640   
1659   434486923  5637169639   
1660   434486921  5637169637   
1661   434486918  5637169633   
1662   434486917  5637169632   
1663   434410014  5637319204   
1664   434486915  5637169629   
1665   434486913  5637169628   
1666   434486911  5637169625   
1667   434486908  5637169623   
1668   434486906  5637169621   
1669   434486903  5637169617   
1670   434486900  5637169613   
1671   434486899  5637169612   
1672   434486894  5637169603   
1673   434486893  5637169601   
1674   434486892  5637169600   
1675   434486888  5637169596   
1676   434486885  5637169593   
1677   434486883  5637169591   
1678   434486880  5637169584   
1679   434486879  5637169583   
1680   434486877  5637169581   
1681   434486875  5637169580   
1682   434486874  5637169579   
1683   434486870  5637169575   
1684   434486869  5637169574   
1685   434486867  5637169570   
1686   434486863  5637169565   
1687   434486862  5637169564   
1688   434486861  5637169563   
1689   434486860  5637169562   
1690   434486858  5637169560   
1691   434486857  5637169559   
1692   434486853  5637169556   
1693   434486850  5637169553   
1694   434486848  5637169551   
1695   434486847  5637169549   
1696   434486846  5637169548   
1697   434486841  5637169543   
1698   434486840  5637169541   
1699   434486839  5637169539   
1704   434403065  5637281768   
1705   540456505  5637521683   
1706   540456504  5637521682   
1708   540462009  5637521750   
1709   540461485  5637521746   
1710   540461479  5637521741   
1711   540461478  5637521740   
1712   540462002  5637521732   
1713   540461997  5637521728   
1714   540461996  5637521727   
1715   540461469  5637521702   
1716   540461468  5637521701   
1717   540461467  5637521700   
1718   540461464  5637521697   
1719   540461983  5637521693   
1720   540456509  5637521686   
1721  

In [52]:
envelope['envelope_matches']=envelope['envelope_matches'].str.get(0)
envelope['envelope_matches']=envelope['envelope_matches'].apply(lambda x:x.lower())
envelope[['length', 'width']]=envelope['envelope_matches'].str.split(" x ",expand=True,)

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\"','',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\"','',str(measure)))
envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
envelope['width']=envelope['width'].apply(lambda quote: quote.replace('"', ''))
envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('-',''))
envelope['width']=envelope['width'].apply(lambda quote: quote.replace('-',''))
envelope['length']=envelope['length'].astype(int)
envelope['width']=envelope['width'].astype(int)
envelope

In [184]:
matches=height[height['matches'].astype(str)!='[]']
matches[curation_col]=height['matches'].astype(str)
matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" H',' in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'H','in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'Inch',' in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'12"','12 in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'in',' in',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))

In [192]:
matches=matches[(matches['external_id'].astype(str)!='69592262')&(matches['external_id'].astype(str)!='65277614')
       &(matches['external_id'].astype(str)!='47401167')
       &(matches['external_id'].astype(str)!='69671877')
       &(matches['external_id'].astype(str)!='41637418')
       &(matches['external_id'].astype(str)!='65277607')
       &(matches['external_id'].astype(str)!='41971185')
       &(matches['external_id'].astype(str)!='65977835')
       &(matches['external_id'].astype(str)!='47001480')
       &(matches['external_id'].astype(str)!='61470569')
       &(matches['external_id'].astype(str)!='47444829')
       &(matches['external_id'].astype(str)!='44283988')
       &(matches['external_id'].astype(str)!='69689885')
       &(matches['external_id'].astype(str)!='60988829')
       &(matches['external_id'].astype(str)!='63571479')
       &(matches['external_id'].astype(str)!='17604490')
       &(matches['external_id'].astype(str)!='61470576')
       &(matches['external_id'].astype(str)!='41637333')]
len(matches)

851

In [206]:
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"101\/2",'10.5',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"325\/8",'32.68',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"17\.0",'17',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\-7\/8",'.88',str(measure)))

matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\-3\/4",'.75',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\-1\\2",'.5',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"1\\5",'0.2',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\-1\/4",'.25',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"91/2",'9.5',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"1/2",'.5',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\-",'',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"3/4",'.75',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"1/5",'0.2',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"1/4",'0.25',str(measure)))
# matches[curation_col].explode().value_counts()

In [228]:
matches

product_id external_id  \
9      1218638256    69716313   
26      817598130    63056167   
31      817187395    62391290   
32      988527092    62391283   
52      988794966    61940949   
...           ...         ...   
77358   817469598    60135025   
77438   817676191    67231690   
77530   817503192    44609955   
77780   817676055    67219315   
77872  1239683305    69679298   

                                                              product_name  \
9                                           Belmar tall Chest Windsor Grey   
26                               Soho Baby Hampton Chifferobe in Stonewash   
31                      Baxton Studio Beacon TV Media Stand in Light Brown   
32                     Baxton Studio Cardiff TV Media Stand in Light Brown   
52                          Zuo® Dresden Dining Chairs in Black (Set of 2)   
...                                                                    ...   
77358           Pacific Play Tents Day Dreamer Sleeping Bag in Blue/Orange   
77438                 Moe's Home Collection Atomic Table Top Decor in Gold   
77530     Southern Enterprises Chicago 24-Inch Floating Shelf in Chocolate   
77780  Moe's Home Collection Propeller Metal Wall Decor in Gold (Set of 2)   
77872                  60-Inch Industrial Wood Shelf with Bracket in Brown   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         long_desc  \
9      The evolur Belmar Tall Chest epitomizes relaxed, coastal style. The unique design features 4 pull-out drawers with 3 open shelves hidden behind a sliding barn door. Soft lines and an antique brass pull give this nursery essential a rustic look. SAFETY TIP: Don't forget to secure your chest to the wall to prevent tipping. Kiln-dried hardwood and veneers Non-toxic rustic grey finish Soft, clean lines and sliding barn door provide charming, coastal charm Antique brass pull Drawers feature 5-piece independent drawer box construction with English dovetail joints and heavy duty side-mounted 3 tier full extension glides Includes anti-tip kit to secure item to the wall Maximum weight capacity: 50 lb. Measures 38\" L x 21\" W x 48\" H Weighs 120 lb. 4 drawers with 3 open shelves behind a sliding barn door No assembly required Meets and exceeds ASTM safety standards 1-year limited manufacturer's warranty Imported Model 886-RG Coordinate with other evolur Belmar Furniture Collection pieces for a complete look (all sold separately)   
26                                                                                                                                                                                                                                                                                                                                  Furnish your nursery in a modern, rustic appearance with the Soho Baby Hampton Chifferobe. Classic clean lines, 5 drawers and a door with adjustable shelving sit atop sturdy square feet, making this stylish storage piece deliver years of usage. 5 side drawe

In [220]:
na_upload=height[height['matches'].astype(str)=='[]']
regex_pattern_na=r'(\d*\-Inch)|()'
na_upload['matches'] = na_upload['product_name'].apply(lambda x: re_extract(regex_pattern_na, x))
na_upload=na_upload[na_upload['matches'].astype(str)=='[]']
# height['matches']= height['matches'].apply(lambda x: sorted(x, key=natural_key))
len(na_upload)

5733

# foot is pulling in inches with two single quotes

In [215]:
foot=na_upload
regex_pattern_ft=r'(\d*\'\s?H)|()'
foot['matches'] = foot['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
foot=foot[foot['matches'].astype(str)!='[]']
foot.head(1)

product_id external_id  \
74   817176151    47360921   

                                                                               product_name  \
74  National Tree 7.5-Foot Frosted Colorado Spruce Pre-Lit Christmas Tree with Clear Lights   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
74  This Frosted Colorado Spruce Pre-Lit Christmas Tree from National Tree has a fresh-cut look thanks to Feel Real™ branch tips finished with life-like frost making it all the more festive. Hinged branches make set-up quick and easy. The National Tree 7.5-Foot Frosted Colorado Spruce Pre-Lit Christmas Tree with Clear Lights features Feel Real branches with a frosted finish For indoor use only Lush and life-like Feel Real branch tips for realism Hinged branches for easy set-up Extra bulbs included 750 lights Tip count: 2187 Measures 7.5' H x 63\" at base Metal stand PE/PVC to emulate spruce UL listed Imported   

    customer_id attributes                                   buckets  \
74            5       None  Trees, Shrubs, Topiaries, & Other Plants   

    bucket_id values matches name_matches  
74       8072   None  [5' H]           []

In [218]:
matches_ft=foot[foot['matches'].astype(str)!='[]']
matches_ft[curation_col]=foot['matches'].astype(str)
matches_ft[curation_col]=matches_ft[curation_col].apply(lambda x: f'"{x}"')
matches_ft[curation_col]=matches_ft[curation_col].apply(lambda measure: re.sub(r"\'\'",'"',str(measure)))
matches_ft[curation_col]=matches_ft[curation_col].apply(lambda measure: re.sub(r"\' H",' ft',str(measure)))

matches_ft[curation_col]=matches_ft[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches_ft[curation_col]=matches_ft[curation_col].apply(lambda measure: re.sub(r'ft',' ft',str(measure)))
matches_ft[curation_col]=matches_ft[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches_ft.head(1)

product_id external_id  \
74   817176151    47360921   

                                                                               product_name  \
74  National Tree 7.5-Foot Frosted Colorado Spruce Pre-Lit Christmas Tree with Clear Lights   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
74  This Frosted Colorado Spruce Pre-Lit Christmas Tree from National Tree has a fresh-cut look thanks to Feel Real™ branch tips finished with life-like frost making it all the more festive. Hinged branches make set-up quick and easy. The National Tree 7.5-Foot Frosted Colorado Spruce Pre-Lit Christmas Tree with Clear Lights features Feel Real branches with a frosted finish For indoor use only Lush and life-like Feel Real branch tips for realism Hinged branches for easy set-up Extra bulbs included 750 lights Tip count: 2187 Measures 7.5' H x 63\" at base Metal stand PE/PVC to emulate spruce UL listed Imported   

    customer_id attributes                                   buckets  \
74            5       None  Trees, Shrubs, Topiaries, & Other Plants   

    bucket_id values matches name_matches  Q:height  
74       8072   None  [5' H]           []  ["5 ft"]

In [221]:
na_upload
regex_pattern_ft=r'(\d*\'\s?H)|()'
na_upload['matches'] = na_upload['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
na_upload=na_upload[na_upload['matches'].astype(str)=='[]']
len(na_upload)

5671

In [224]:
na_upload[na_upload['buckets'].astype(str)!='TV Stands/Media Consoles'][0:500]

product_id external_id  \
61     817468676    47538207   
75     817205127    47307643   
76        884783    47091467   
93    1199642759    43421497   
97     817164179    43105311   
98     817164203    43105250   
99     817168228    44842819   
100    817168227    44842345   
101    817168221    44842253   
102    817462532    44842239   
105    817461541    44252168   
108    817458034    41637265   
109    817458029    41637203   
111    817457662    41340004   
114    817459326    42893844   
119    817160981    41191538   
139    817157137    18006618   
140    817481133    17291483   
141    817454588    17291335   
142    817481139    17291203   
147    817210588    62488211   
153    817476860    62649759   
160   1212509858    69717889   
161   1214062035    69717851   
170   1249468899    69709851   
223   1144567925    69690728   
225    817453160    13525200   
228   1054428335    69679438   
266    988766613    69633555   
267    988766605    69633644   
268    988766595    69633639   
269    988766544    69633511   
270   1066124245    69633599   
271   1066124239    69633596   
272    988766485    69633700   
273   1110733151    69632726   
274    988766375    69632717   
322   1194597027    69622478   
331   1006759080    69616238   
332   1006759034    69616347   
335   1245322142    69615468   
390    988475591    69591139   
410   1083591401    69589231   
417   1083588959    69537282   
422    988758368    69583690   
427    817451871    69540638   
428    988753898    69539396   
430    817421428    69536644   
456    817416430    69515516   
457    988749277    69513236   
472    817413471    69506018   
473    817449688    69503202   
474    817658882    69503338   
475    817449034    69503250   
476    817658419    69503285   
489    817632730    69496298   
550    817624745    69287060   
552    817624669    69292217   
553    817400231    69292323   
592    817394008    69093692   
608   1058312541    69679388   
609   1058312538    69679385   
610   1058312535    69679382   
710    817384238    68789190   
711    817442077    68542719   
736    817378958    68622329   
739    817441883    68601195   
740    817441901    68601836   
741   1171985659    68602673   
742   1171985655    68602918   
743   1171985600    68602604   
769   1171985582    68602680   
770   1171985527    68602635   
772   1171985359    68582579   
773    817441784    68543464   
774    817441830    68544355   
775    817441813    68544379   
776    817441805    68544461   
791    817608320    68436575   
795    817442085    68542818   
796    817442074    68542894   
797    817441803    68543662   
798    817442088    68543167   
799    817441760    68544485   
800    817441804    68543747   
801    817441776    68544249   
811    817606837    68296391   
816    817373706    68265908   
819    988710228    68253851   
827    817444080    69017421   
867   1240719720    68234478   
868   1240719662    68234492   
893    817370397    68167455   
939    817365604    67898619   
1005   817354574    67356010   
1016   817598284    67681785   
1019   817361453    67649709   
1030   817589289    67117444   
1031   817589285    67117413   
1058   817583581    66755333   
1059   817582691    66693345   
1067  1224165086    66604969   
1073   817340796    66714767   
1094   817576998    66302247   
1253  1200479864    65564165   
1255   817318668    65692424   
1256   817567328    65685259   
1257   817645197    65665176   
1261   817564756    65506172   
1273   817562538    65346617   
1274   817562520    65346556   
1278   988644171    65174913   
1295   817644537    65246948   
1305   817303880    64873695   
1367   817295096    64274928   
1441   817542977    63618808   
1442   988625170    63619423   
1444   817283183    63597127   
1471   817282604    63556988   
1516   817543443    63662382   
1709   817271583    63257823   
1729   817529949    62251891   
1731   817265052    62251792   
1732   817529930    62251747   
1733  

# Seperate Bukets and start curating

# Wall Sculptures

In [7]:
wall=df[df['buckets'].astype(str)=='Wall Sculptures & Dimensional Art']
regex_pattern_wall = r"(?i)(\d*\.?\d*.?Inch\s?x\s?\d*\.?\d*.?Inch)|(\d*.?Inch)|(\d*\.\d*.?Inch)|()" 
wall['matches'] =wall['long_desc'].apply(lambda x: re_extract(regex_pattern_wall, x))
wall['name_matches'] =wall['product_name'].apply(lambda x: re_extract(regex_pattern_wall, x))
wall_df=wall[wall['name_matches'].astype(str)!='[]']

# Square

In [8]:
wall_df['name_matches']=wall_df['name_matches'].str.get(0)
wall_df[['length', 'width']]=wall_df['name_matches'].str.split("x",expand=True,)
wall_df.head(1)

product_id external_id  \
145  1230264608    69724775   

                                                                       product_name  \
145  Glitzhome® 24.25-Inch x 11.25-Inch Whimsical Metal Watermelon Gecko Wall Decor   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
145  Add a fun look to your wall with this Whimsical Metal Watermelon Gecko Wall Decor from Glitzhome. Crafted of sturdy iron to last, this colorful wall decor is ready to hang and displays a cute gecko with watermelon coloring and big pink eyes. Metal gecko shape Spring neck Displays watermelon design elements for a cute look Hand painted glossy finish Appropriate for indoor/outdoor use Iron Ready to hang Weighs 1 lb. 1-year limited manufacturer's warranty Imported   

     customer_id attributes                            buckets  bucket_id  \
145            5       None  Wall Sculptures & Dimensional Art       8122   

    values matches             name_matches       length        width  
145   None      []  24.25-Inch x 11.25-Inch  24.25-Inch    11.25-Inch

In [9]:
wall_df['length']=wall_df['length'].apply(lambda measure: re.sub(r"\-Inch",'',str(measure)))
wall_df['width']=wall_df['width'].apply(lambda measure: re.sub(r"\-Inch",'',str(measure)))  
wall_df['length']=wall_df['length'].apply(lambda measure: re.sub(r" ",'',str(measure)))
wall_df['width']=wall_df['width'].apply(lambda measure: re.sub(r" ",'',str(measure))) 
wall_df['length']=wall_df['length'].astype(str)
wall_df['width']=wall_df['width'].astype(str)

In [10]:
wall_df['compared']=wall_df['length'].equals(wall_df['width'])
wall_df.loc[(wall_df['length']==wall_df['width']), 'compared']=wall_df['length']
wall_new=wall_df[wall_df['compared'].astype(str)!='False']
wall_new[curation_col]=wall_new['compared']   
wall_new[curation_col]=wall_new[curation_col].apply(lambda x: x+' in')
wall_new.head(1)

In [69]:
len(wall_new)

67

In [12]:
square=wall_df[wall_df['width'].astype(str)=='None']
square[curation_col]=square['length']
square[curation_col]=square[curation_col].apply(lambda x: x+' in')
square.head(1)

product_id external_id  \
240  1016177218    69674934   

                                                 product_name  \
240  28.5-Inch Metal Wind Spinner Wall Decor in Vintage White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
240  Crafted of galvanized iron, the Wind Spinner Wall Décor adds a farmhouse touch to any wall. With quiet vintage appeal, this 3D fan shaped design has rivets and flanging on both sides for added depth, and a rustic look. Adds farmhouse charm to any home décor 3D fan-shaped design with rivets added on Flanging on both sides give the decor depth Vintage red, vintage white and grey painted finishes 100% iron Ready to hang with gourd hole on the back Measures 22\" diameter or 28.5\" diameter Imported   

     customer_id attributes                            buckets  bucket_id  \
240            5       None  Wall Sculptures & Dimensional Art       8122   

    values matches name_matches length width compared Q:height  
240   None      []    28.5-Inch   28.5  None    False  28.5 in

In [68]:
len(square)

244

# Trees

In [43]:
trees=df[df['buckets'].astype(str)=='Trees, Shrubs, Topiaries, & Other Plants']
# \d*.?Foot
regex_pattern_tree = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|()" 
trees['matches'] =trees['long_desc'].apply(lambda x: re_extract(regex_pattern_tree, x))
trees['name_matches'] =trees['product_name'].apply(lambda x: re_extract(regex_pattern_tree, x))
trees_df=trees[trees['name_matches'].astype(str)!='[]']

In [46]:
#import regex as re
matches_tree=trees_df
matches_tree[curation_col]=matches_tree['name_matches'].astype(str)
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda x: f'"{x}"')
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\-Foot",' ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'\-foot',' ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"\-1\/2",'.5',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r' 1\/2','.5',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r" Foot",' ft',str(measure)))

matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'7 ft 6\-Inch','7.5 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"6 ft 5\-Inch",'6.4 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'4 ft 6\-Inch','4.6 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"7 ft 5\-Inch",'7.4 ft',str(measure)))
matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r"2 ft 5\-Inch",'2.4 ft',str(measure)))

matches_tree[curation_col]=matches_tree[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches_tree.head(10)

product_id external_id  \
60     817488197    47362154   
77    1199645354    45485725   
78     817169202    45485633   
1606  1174458301    66735656   
2413   817266290    62508643   
2414   817266252    62510691   
2415   817266253    62505772   
2416   817530699    62507837   
2418   817530639    62505987   
2419   817530590    62486071   

                                                                                        product_name  \
60                            National Tree Company 3-Foot Pine Cone Tree with Warm White LED Lights   
77                           Fraser Hill Farm 7.5-Foot Southern Peace Pine Artificial Christmas Tree   
78                    Fraser Hill Farm 7.5-Foot Hillside Slim Flocked Pine Artificial Christmas Tree   
1606                National Tree Company Dual Color® 4.5-Foot Pre-Lit Newbury Spruce Christmas Tree   
2413                          National Tree Company 7-Foot Fraser Slim Fir Artificial Christmas Tree   
2414    National Tree Company 7-1/2-Foot Pre-Lit North Valley White Spruce Artificial Christmas Tree   
2415    National Tree Company 4-1/2-Foot Pre-Lit North Valley White Spruce Artificial Christmas Tree   
2416                           National Tree Company 3-Foot Kincaid Spruce Artificial Christmas Tree   
2418  National Tree Company® 6.5-Foot Downswept Douglas Pencil Slim Christmas Tree with Clear Lights   
2419                National Tree Company 6-1/2-Foot Pre-Lit Carolina Pine Artificial Christmas Tree   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         long_desc  \
60                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                The festive Pine Cone Holiday Collection from National Tree Company brings the beauty of the holiday season into your home. These gorgeous holiday pieces bedecked with decorative pinecones and can be displayed indoors or covered outdoor areas. National Tree Company Pine Cone Holiday Collection offers attractive festive pieces to decorate your home indoors or outdoors PVC construction trimmed with decorative pine cones For indoor or covered outdoor use Imported   
77                                                                                                                              

In [47]:
matches_tree[curation_col].explode().value_counts()

["7.5 ft"]    44
["9 ft"]      34
["6.5 ft"]    26
["4.5 ft"]    15
["12 ft"]     13
["3 ft"]      13
["6 ft"]      12
["4 ft"]      11
["5 ft"]      10
["7 ft"]       8
["10 ft"]      5
["2 ft"]       5
["16 ft"]      3
["5.5 ft"]     3
["14 ft"]      2
["15 ft"]      1
["6.4 ft"]     1
["3.5 ft"]     1
["8 ft"]       1
["2.4 ft"]     1
["4.6 ft"]     1
["7.4 ft"]     1
Name: Q:height, dtype: int64

In [67]:
len(matches_tree)

211

# Stools

In [50]:
stools=df[df['buckets'].astype(str)=='Stools']
regex_pattern_stool = r"(?i)(\d*\.\d*.?Inch)|(\d*.?Inch)|(\d*.?.?H)|()" 
stools['matches'] =stools['long_desc'].apply(lambda x: re_extract(regex_pattern_stool, x))
stools['name_matches'] =stools['product_name'].apply(lambda x: re_extract(regex_pattern_stool, x))
stools_df=stools[stools['name_matches'].astype(str)!='[]']

In [51]:
#import regex as re
matches_stool=stools_df
matches_stool[curation_col]=matches_stool['name_matches'].astype(str)
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda x: f'"{x}"')
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r"\-Inch",' in',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r" Inch",' in',str(measure)))
matches_stool[curation_col]=matches_stool[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches_stool.head(10)

product_id external_id  \
1653   817330983    66220954   
1755   988656011    66027362   
2579   817259945    61757912   
2580   817259946    61757936   
2874   817229475    45421525   
2875   817229476    45421549   
2926   988579392    18776588   
2944   988574450    40527833   
4161  1244996285    66220183   
6394   817216802    18078465   

                                                                      product_name  \
1653                                        Linon Home Mossy Oak 45-Inch Bar Stool   
1755              American Woodcrafters Taranto 34-Inch Leather Bar Stool in Brown   
2579    Carolina Cottage Remick 24-Inch Barley Twist Leg Counter Stool in Espresso   
2580  Carolina Cottage Remick 30-Inch Barley Twist Leg Bar Stool in Espresso/Cream   
2874                               Westwood Patches 24-Inch Counter Stool in White   
2875                                             Westwood 24-Inch Bar Stool in Red   
2926                      Crosley X-Back 24-Inch Counter Stool in Black (Set of 2)   
2944                                           Lakewood 30-Inch Bar Stool in Brown   
4161                             Linon Home Westwood 26-Inch Counter Stool in Grey   
6394                                               Hillsdale Jenkins 27-Inch Stool   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1653                                                                                                                                                                                                                                                                              Add a rustic look to your home with the Linon Mossy Oak Bar and Counter Stools. Featuring a dark walnut finish and a unique nature inspired print upholstery, these classic stools have four foot rails for added comfort and stability. Adds rustic style to your home counter or bar Dark walnut finish is accented with hand brushed black strokes for depth Plush padded upholstery Unique Mossy Oak Nativ Living print Four footrails add stability and durability to the stool Wood/fabric/foam Spot clean 275 lb. weight capacity 6-year limited manufacturer's warranty Imported   
1755                                                                                                                                                                                                                                                                                                                      The Taranto Stool features industrial flare with its charcoal finished steel back and solid wood cap accent. The washed brown finish and flared legs add style and are accented by the black bonded leather seat and charcoal finished nailhead trim. A 360 degree swivel keeps you in the conversation while the circular footrest ensures your comfort Stylish charcoal finished nail head accents Assembly required Leather/foam seat Wood frame construction 250 lb. maximum weight capacity 1-year limited manufacturer's warranty Imported   
2579                                                           Featuring spiral barley-twist legs, soft cotton-linen upholstery and brass nailhead trim, the striking Re

In [52]:
matches_stool[curation_col].explode().value_counts()

["30 in"]       25
["24 in"]       19
["26 in"]        5
["29 in"]        5
["25 in"]        3
["31 in"]        3
["32 in"]        2
["34 in"]        2
["44.5 in"]      2
["26.4 in"]      2
["26.5 in"]      1
["27 in"]        1
["38.75 in"]     1
["41 in"]        1
["39 in"]        1
["18 in"]        1
["20 in"]        1
["30.5 in"]      1
["45 in"]        1
["24.5 in"]      1
Name: Q:height, dtype: int64

In [66]:
len(matches_stool)

78

# Side Tables

In [61]:
side_tables=df[df['buckets'].astype(str)=='Side/End Tables']
regex_pattern_table = r"(?i)(\d*\.\d*”.?H)|(\d*\.\d*.?Inch)|(\d*.?Inch)|(\d*”.?H)|()" 
side_tables['matches'] =side_tables['long_desc'].apply(lambda x: re_extract(regex_pattern_table, x))
side_tables['name_matches'] =side_tables['product_name'].apply(lambda x: re_extract(regex_pattern_table, x))
side_tables_df=side_tables[side_tables['matches'].astype(str)!='[]']
side_tables_df['matches']= side_tables_df['matches'].apply(lambda x: sorted(x, key=natural_key))

In [62]:
#import regex as re
matches_side_table=side_tables_df
matches_side_table[curation_col]=side_tables_df['matches'].astype(str)
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda x: f'"{x}"')
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'\” H',' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'\”H',' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r" ",'',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r"in",' in',str(measure)))
matches_side_table[curation_col]=matches_side_table[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))

In [65]:
len(matches_side_table)

90

# Sculptures and figurines

In [126]:
sculptures=df[df['buckets'].astype(str)=='Sculptures, Figurines, & Decorative Objects']
# \d*.?Foot
regex_pattern_sc = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|(\d*\.\d*”.?H)|(\d*\.\d*.?Inch.?H)|(\d*.?Inch.?H)|(\d*”.?H)|()" 
sculptures['matches'] =sculptures['long_desc'].apply(lambda x: re_extract(regex_pattern_sc, x))
sculptures['name_matches'] =sculptures['product_name'].apply(lambda x: re_extract(regex_pattern_sc, x))
sculptures_df=sculptures[(sculptures['matches'].astype(str)!='[]')]
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='69241383']
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='16623601']
sculptures_df['matches']= sculptures_df['matches'].apply(lambda x: sorted(x, key=natural_key))
# sculptures_df

In [128]:
regex_pattern_sc_mis = r"(?i)(\b\d*.?Foot\s?\d+.?Inch)|(\b\d*.?\d*\/\d*.?Foot)|(\b\d*\.?\d*.?Foot)|(\b\d*.?Foot)|(\d*\.\d*''.?H)|(\d*\.\d*.?Inch.?H)|(\d*.?Inch.?H)|(\d*''.?H)|()" 
sculptures['mis_matches'] =sculptures['long_desc'].apply(lambda x: re_extract(regex_pattern_sc_mis, x))
sculptures['misc_name_matches'] =sculptures['product_name'].apply(lambda x: re_extract(regex_pattern_sc_mis, x))
sculptures_df=sculptures[(sculptures['misc_name_matches'].astype(str)!='[]')]
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='69241383']
sculptures_df=sculptures_df[sculptures_df['external_id'].astype(str)!='16623601']
sculptures_df['matches']= sculptures_df['matches'].apply(lambda x: sorted(x, key=natural_key))
sculptures_df

product_id external_id  \
6404   1027974529    65988633   
23624  1196481650    65994351   
25008  1011989107    69406447   
38221  1174818157    69606258   
61918  1071790838    69406430   
85552   988764235    69623246   

                                                                              product_name  \
6404      5-Foot Standing Santa Claus Christmas Figure with Flocked Alpine Tree and Wreath   
23624                                Vickerman 6-Foot Plush Scarecrow Decoration in Purple   
25008  Northlight 5-Foot Snowflake Silhouette LED Rope Light Christmas Decoration in White   
38221                           5-Foot Skeleton Unicorn Inflatable LED Lit Lawn Decoration   
61918  Northlight 4-Foot Snowflake Silhouette LED Rope Light Christmas Decoration in White   
85552                                                Mr. Pumpkin 3-Foot Metal Candy Holder   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
6404                                                                                                                                                                                                                                                                                            Santa Claus is coming to town. Bring a rustic feel to your home this Christmas season with this standing Santa Claus figure. Wearing a plaid fur-trimmed coat, hat and matching boots, he is a unique but festive touch to your holiday decor. Smiling Santa wearing glasses Santa is wearing a red and black coat with brown faux fur trim He is carrying a wreath in one hand and a flocked tree in the other Fabric/plastic/faux fur/wood Measures 5' H x 22\" W x 24\" D Imported   
23624                                                                                                                                                                                                                                                          Add a classic, festive accent to your home with Vickerman's 6' Plush Autumn Scarecrow. Perfect to place in your home for the Fall season, this scarecrow can easily stand up with the PVC pole frame or be adjusted into a sitting position. Stands 6 feet tall! Easy assembly required Plush parts connect with PVC poles to make the scarecrow stand up or sit down Scarecrow's feet are connected to a 16\" x 16\" display platform This is a display figure it is not intended to be used as a toy Indoor use only   
25008             This Northlight Snowflake Silhouette LED Rope Light Christmas Decoration is perfect for adding seasonal charm to your space for the holidays. Ready-to- hang snowflake has a dual sided design with clear LED lights that cast a cozy, festive glow. Illuminated dual-sided snowflake silhouette 1.5\" bulb spacing Suitable for indoor/outdoor use 2' snowflake features: 120 clear LED bulbs Measures 24' L x 24\" W Weighs 2 lb. 4' snowflake features: 216 clear LED bulbs Measures 48\" L x 48\" W Weighs 2.8 lb. 5' snowflake features: 264 clear LED bulbs Measures 60\" L x 60\" W Weighs 2.5 lb Plastic/metal/ wire Plugs into standard outlets Hangable design No assembly required White lead cord measures 6' L UL listed Imported Each sold separately   
38221  Add spooky Halloween fun and whimsy to your yard with this Skelet

In [121]:
#import regex as re
matches_sc=sculptures_df
matches_sc[curation_col]=sculptures_df['matches'].astype(str)

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda x: f'"{x}"')
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'\”','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'H',' in',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'  ',' ',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'Inch H',' in',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'Inch','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"\-",'',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r' ','',str(measure)))
matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"in",' in',str(measure)))

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'1 in","16 in','16 in',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"6 ft 5\-Inch",'6.4 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'4 ft 6\-Inch','4.6 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"7 ft 5\-Inch",'7.4 ft',str(measure)))
# matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r"2 ft 5\-Inch",'2.4 ft',str(measure)))

matches_sc[curation_col]=matches_sc[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_sc.head(10)

In [122]:
matches_sc[curation_col].explode().value_counts()

["9 in"]                      4
["14 in"]                     3
["8 in"]                      3
["11 in"]                     3
["12 in"]                     2
["23 in"]                     2
["16 in"]                     2
["5 in"]                      2
["19.5 in"]                   1
["14.5 in"]                   1
["26 in"]                     1
["9 in","10 in"]              1
["19 in"]                     1
["18.5 in"]                   1
["3 in","4 in","5 in"]        1
["20 in"]                     1
["17 in"]                     1
["5.5 in","7 in","8.5 in"]    1
["9.5 in","10 in"]            1
["12.75 in","13 in"]          1
["9.5 in"]                    1
["6.25 in","6 in"]            1
["32 in"]                     1
["13 in"]                     1
["7 in"]                      1
["16 in","17 in"]             1
["12 in","14 in"]             1
["21 in"]                     1
["28 in"]                     1
["15 in"]                     1
["16.5 in"]                   1
["10.75 

In [104]:
matches_sc[matches_sc['matches'].astype(str)==' foot']

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, matches, name_matches, Q:height]
Index: []

In [95]:
len(matches_sc)

52

# foot

In [ ]:
# regex_pattern = r"(?i)(\b\d+\.\d*.?\"\s?H\s)|(\b\d+.?\"\s?H)|(\b\d+\.\d*.?\"\s?H)|(\d*\s?\-\d*\/\d*\\\"\s?H)|(\d*\s\-?\d*\/\d*\\\"\s?H)|()" 
# regex_pattern_diam = r'(?i)(\d+\.\d*\s?\\\"\s?i?n?\s?diam)|(\d*\s?\\\"\s?i?n?\s?diam)|()'
# regex_pattern_foot = r"(?i)(\b\d+\.\d*\'\s?H\s)|(\b\d+.?\'\s?H)|(\b\d+\.\d*\'\s?H)|(\d*\s?\-\d*\/\d*\\?\'\s?H)|(\d*\s\-?\d*\/\d*\\\'\s?H)|()" 

In [ ]:
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# df['diam_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diam, x))
# df['foot_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_foot, x))
# len(df)

In [ ]:
# foot=df[df['foot_matches'].astype(str)!='[]']
# foot=foot[foot['buckets'].astype(str)=='Trees, Shrubs, Topiaries, & Other Plants']
# len(foot)

In [ ]:
# # x=df[df['matches'].astype(str)!='[]']
# x=df
# len(x)
# # len(x)

In [ ]:
# x=x[x['matches'].astype(str)!='[]']

In [ ]:
# replacement_foot = {
#     "\' H": " ft",
#     "' H": "ft",
#     '  ft': " ft",
#     "\'H": " ft",
#     "' h": " ft",
#     "3\' H": "3 ft"
# }

In [ ]:
# # Replacement
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: [replace_all(string, replacement_foot) for string in x] if type(x)==list else x)

In [ ]:
# # Replacement
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: [replace_all(string, replacement_foot) for string in x] if type(x)==list else x)
# # foot.head(3)

In [ ]:
# foot['foot_matches']= foot['foot_matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# # foot.head(3)

In [ ]:
# #import regex as re
# matches_foot=foot
# matches_foot[curation_col]=matches_foot['foot_matches'].astype(str)
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda x: f'"{x}"')
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_foot.head(3)

In [ ]:
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r"\-1\/2",'.5',str(measure)))
# matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'\"\\xa0H',' ft',str(measure)))

In [ ]:
# matches_foot[curation_col].explode().value_counts()

In [ ]:
replacement = {
    '\" H ': " in",
    "\ in": " in",
    '\" H': " in",
    '\" H': " in",
    '\" H ': " in",
    "\ in": " in",
    '\" H': " in",
    '\" H': " in",
    '\" h': " in",
    '\"H': " in"
}

In [ ]:
x=x[x['matches'].astype(str)!='[]']
x.head(1)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(1)

In [ ]:
x['matches']= x['matches'].apply(
    lambda x: sorted(x, key=natural_key))
x.head(1)

In [ ]:
# x=x[(x['matches'].astype(str)!='3/4 in')&(x['matches'].astype(str)!='1/4 in')&(x['matches'].astype(str)!='1/5 in')&(x['matches'].astype(str)!=' 3/4 in')]

In [ ]:
# replacement_diam = {
#     'diam': "",
#     " ": "",
#     '\"': "",
#     'in': "",
#     'Diam': ""
# }
# # Replacement
# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: [replace_all(string, replacement_diam) for string in x] if type(x)==list else x)

# # Replacement
# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: [replace_all(string, replacement_diam) for string in x] if type(x)==list else x)
# diam.head(3)

# diam['diam_matches']= diam['diam_matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# diam.head(3)

# matches

In [ ]:
# matches=x

In [ ]:
# #import regex as re
# matches[curation_col]=matches['matches'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches.head(3)

In [ ]:
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1\/2','.5',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3\/4','.75',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3\/4','.75',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1\/2','.5',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1\/4','.25',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1\/4','.25',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3\/8','.38',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3\/8','.38',str(measure)))

# matches=matches[matches[curation_col].astype(str)!='["3/4 in"]']
# matches=matches[matches[curation_col].astype(str)!='["1/4 in"]']
# matches=matches[matches[curation_col].astype(str)!='["1/5 in"]']

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-4\/5','.8',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 4\/5','.8',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/5','.2',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/5','.2',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-5/8','.62',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 5/8','.62',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-2/3','.66',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 2/3','.66',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3/10','.3',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3/10','.3',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/3','.33',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/3','.33',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-8/9','.89',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 8/9','.89',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-3/5','.6',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 3/5','.6',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-2/5','.4',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 2/5','.4',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-7/10','.7',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 7/10','.7',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/10','.1',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/10','.1',str(measure)))

# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-1/8','.13',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 1/8','.13',str(measure)))


# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\-7/8','.88',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r' 7/8','.88',str(measure)))

In [ ]:
# # matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'1575 in','15.75 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\\xa0','',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'4563 in','45.63 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'3625 in','36.25 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.625','.63',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.375','.38',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\.875','.88',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'335 in","635 in','33.5 in","63.5 in',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'45 in","395 in','45 in","39.5 in',str(measure)))

In [ ]:
# matches[matches[curation_col].astype(str)=='["0.59 in"]']

In [ ]:
# matches[matches[curation_col].astype(str)=='["45 in","395 in"]']

In [ ]:
len(matches)

In [ ]:
# matches_diam=diam

In [ ]:
# #import regex as re
# matches_diam[curation_col]=matches_diam['diam_matches'].astype(str)
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda x: f'"{x}"')
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches_diam.head(3)

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\\\\',' in',str(measure)))

In [ ]:
# matches_diam=matches_diam[matches_diam['buckets'].astype(str)!='Trees, Shrubs, Topiaries, & Other Plants']
# matches_diam=matches_diam[matches_diam['buckets'].astype(str)=='Hanging Accents & Accessories']

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\\',' in',str(measure)))

In [ ]:
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'.875','.88',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'\"10 in\"\,\"10 in\"','"10 in"',str(measure)))
# matches_diam[curation_col]=matches_diam[curation_col].apply(lambda measure: re.sub(r'.725','.73',str(measure)))

In [ ]:
# matches_diam[curation_col].explode().value_counts()[0:500]

In [ ]:
# error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [229]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-height-sculpture - {attribute}-- matches upload_Shawn.csv',index=False)

In [230]:
looks_good('Bed Bath & Beyond', attribute, df, matches)

In [ ]:
def looks_good_na(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-NA - {attribute}-- match upload_Shawn.csv',index=False)

In [ ]:
looks_good_na('Bed Bath & Beyond', attribute, df, na_upload)

In [ ]:
matches_side_table+matches_stool+matches_tree